<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EC%A6%9D%EB%B6%84/L%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화  'Colab Notebooks'   images.zip        labels.zip   val.txt
 classes.txt	  images_1650.zip    labels_1650.zip  'My Drive'


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='L')] #조건에 맞는 데이터 출력
#ts=ts[(ts['digit_1'] =='D')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000099,L,68,682,사업장에서,아파트를 위해,아파트 관리
1,id_0000106,L,68,682,사무실에서,계약의대리,부동산중개
2,id_0000142,L,68,681,사무실에서,비주거용,임대
3,id_0000164,L,68,682,상가사무실에서,빌딩입주자 위해,건물관리서비스
4,id_0000193,L,68,682,사업장에서,아파트를 위해,주거용 아파트 관리서비스
...,...,...,...,...,...,...,...
40135,id_0999952,L,68,682,부동산,중개,부동산중개
40136,id_0999980,L,68,682,사업장에서,고객의요청에의해,부동산중개
40137,id_0999983,L,68,682,사업장에서,예채아파트를,관리
40138,id_0999991,L,68,682,부동산중개소에서,계약및중개등을 통해 수수료받음,부동산 거래중개


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000099,L,68,682,사업장에서 아파트를 위해 아파트 관리
1,id_0000106,L,68,682,사무실에서 계약의대리 부동산중개
2,id_0000142,L,68,681,사무실에서 비주거용 임대
3,id_0000164,L,68,682,상가사무실에서 빌딩입주자 위해 건물관리서비스
4,id_0000193,L,68,682,사업장에서 아파트를 위해 주거용 아파트 관리서비스
...,...,...,...,...,...
40135,id_0999952,L,68,682,부동산 중개 부동산중개
40136,id_0999980,L,68,682,사업장에서 고객의요청에의해 부동산중개
40137,id_0999983,L,68,682,사업장에서 예채아파트를 관리
40138,id_0999991,L,68,682,부동산중개소에서 계약및중개등을 통해 수수료받음 부동산 거래중개


## **# 데이터 증분 및 맞춤법 교정**

In [ ]:
# 맞춤법 검사

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-0_6hj2y6
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-0_6hj2y6
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=e28ffe2df6104f0dfac72789e9ff479e1aa796e7e29453c5985d17a1a6377048
  Stored in directory: /tmp/pip-ephem-wheel-cache-u4jjunb5/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [34]:
from hanspell import spell_checker
import random
import pickle
import re

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)


df['text']=pd.DataFrame(temp_result)  

In [35]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000099,L,68,682,사업장에서 아파트를 위해 아파트 관리
1,id_0000106,L,68,682,사무실에서 계약의 대리 부동산 중개
2,id_0000142,L,68,681,사무실에서 비주거용 임대
3,id_0000164,L,68,682,상가 사무실에서 빌딩 입주자 위해 건물관리 서비스
4,id_0000193,L,68,682,사업장에서 아파트를 위해 주거용 아파트 관리 서비스
...,...,...,...,...,...
40135,id_0999952,L,68,682,부동산 중개 부동산 중개
40136,id_0999980,L,68,682,사업장에서 고객의 요청에 의해 부동산 중개
40137,id_0999983,L,68,682,사업장에서 예체 아파트를 관리
40138,id_0999991,L,68,682,부동산 중개소에서 계약 및 중개 등을 통해 수수료 받음 부동산 거래 중개


In [36]:
df['digit_3'].value_counts()

682    33212
681     6928
Name: digit_3, dtype: int64

In [37]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [44]:
df1=df[df['digit_3']==681] #4개 증분 필요
df2=df[df['digit_3']==682] 

In [45]:
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)

In [46]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df1['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=3)
  id_list=['p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n]), 'p'+str(df1['AI_id'][n])]
  digit_1_list=[df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n], df1['digit_1'][n]]
  digit_2_list=[df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n], df1['digit_2'][n]]
  digit_3_list=[df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n], df1['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df1_to_insert = pd.DataFrame(data_to_insert)
  df1= df1.append(df1_to_insert, ignore_index=True)
  n=n+1

In [47]:
df1 #df1 완성함

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000142,L,68,681,사무실에서 비주거용 임대
1,id_0000214,L,68,681,"택지 농장, 용지 사업장에서 자영 개발 분양 및 산업단지 개발 조성"
2,id_0000389,L,68,681,택지 농지 등을 위탁 개발 분양 판매
3,id_0000430,L,68,681,사업장에서 임대를 통하여 비거주용 건물 임대업
4,id_0000510,L,68,681,사업장에서 임대를 통하여 비거주용 건물 임대업
...,...,...,...,...,...
34635,pid_0999904,L,68,681,사업장에서 임대를 통하여 비거주용 건물 임대업
34636,pid_0999995,L,68,681,비주거용 사무실에서 임대
34637,pid_0999995,L,68,681,사무실에서 비주거용 임대
34638,pid_0999995,L,68,681,비주거용


기존 데이터 합치기

In [50]:
df=pd.concat([df1, df2])

In [51]:
df['digit_3'].value_counts()

681    34640
682    33212
Name: digit_3, dtype: int64

In [52]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000142,L,68,681,사무실에서 비주거용 임대
1,id_0000214,L,68,681,"택지 농장, 용지 사업장에서 자영 개발 분양 및 산업단지 개발 조성"
2,id_0000389,L,68,681,택지 농지 등을 위탁 개발 분양 판매
3,id_0000430,L,68,681,사업장에서 임대를 통하여 비거주용 건물 임대업
4,id_0000510,L,68,681,사업장에서 임대를 통하여 비거주용 건물 임대업
...,...,...,...,...,...
33207,id_0999907,L,68,682,부동산 중개소에서 계약 및 중개 등을 통해 수수료 받음 부동산 거래 중개
33208,id_0999952,L,68,682,부동산 중개 부동산 중개
33209,id_0999980,L,68,682,사업장에서 고객의 요청에 의해 부동산 중개
33210,id_0999983,L,68,682,사업장에서 예체 아파트를 관리


In [53]:
df.to_csv('df_L_inc.csv', index=False, encoding='utf-8-sig')